# Association Rule Based Recommender System

**İş Problemi**

> Aşağıda 3 farklı kullanıcının sepet bilgileri verilmiştir.
> Bu sepet bilgilerine en uygun ürün önerisini birliktelik kuralı kullanarak yapınız. Ürün önerileri 1 tane ya da 1'den fazla olabilir.
> Karar kurallarını 2010-2011 Germany müşterileri üzerinden türetiniz.
> * Kullanıcı 1’in sepetinde bulunan ürünün id'si: 21987
> * Kullanıcı 2’in sepetinde bulunan ürünün id'si : 23235
> * Kullanıcı 3’in sepetinde bulunan ürünün id'si : 22747

*Veri Seti Hikayesi*

Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış işlemlerini içeriyor.
Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi mevcuttur.

8 Değişken, 541.909 Gözlem, 45.6MB

* InvoiceNo: Fatura Numarası ( Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder )
* StockCode: Ürün kodu ( Her bir ürün için eşsiz )
* Description: Ürün ismi
* Quantity: Ürün adedi ( Faturalardaki ürünlerden kaçar tane satıldığı)
* InvoiceDate: Fatura tarihi
* UnitPrice: Fatura fiyatı ( Sterlin )
* CustomerID: Eşsiz müşteri numarası
* Country: Ülke ismi

# Görev 1: Veriyi Hazırlama

In [1]:
# Adım 1: Online Retail II veri setinden 2010-2011 sheet’ini okutunuz.
# !pip install mlxtend
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

# Data pre-prcessing

def missing_values_analysis(data):
    na_columns = [col for col in data.columns if data[col].isnull().sum() > 0]
    n_miss = data[na_columns].isnull().sum().sort_values(ascending=True)
    ratio = (data[na_columns].isnull().sum() / data.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

def check_df(data, row_num=5, col_num=10):
    print("*************** Dataset Shape ***************")
    print("No. of Rows:", data.shape[0], "\nNo. of Columns:", data.shape[1])
    print("*************** Dataset Information ***************")
    print(data.info())
    print("*************** Types of Columns ***************")
    print(data.dtypes)
    print(f"*************** First {row_num} Rows ***************")
    print(data.iloc[:row_num,:col_num])
    print(f"*************** Last {row_num} Rows ***************")
    print(data.iloc[-row_num:,:col_num])
    print("*************** Summary Statistics of The Dataset ***************")
    print(data.describe([0.10, 0.25, 0.50, 0.70, 0.80, 0.90, 0.95, 0.99]).T)
    print("*************** Dataset Missing Values Analysis ***************")
    print(missing_values_analysis(data))




df_ = pd.read_csv("/kaggle/input/online-retail/Online_Retail.csv")
df_ = df_.astype({'InvoiceNo':'string'})
df_ = df_.astype({'StockCode':'string'})
df_ = df_.astype({'Description':'string'})
df_ = df_.astype({'Country':'string'})
df = df_.copy()
check_df(df)

*************** Dataset Shape ***************
No. of Rows: 541909 
No. of Columns: 8
*************** Dataset Information ***************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  string 
 1   StockCode    541909 non-null  string 
 2   Description  540455 non-null  string 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  string 
dtypes: float64(2), int64(1), object(1), string(4)
memory usage: 33.1+ MB
None
*************** Types of Columns ***************
InvoiceNo      string[python]
StockCode      string[python]
Description    string[python]
Quantity                int64
InvoiceDate            object
UnitPrice             float64
CustomerID     

In [3]:
#Data cleaning

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def retail_data_prep(dataframe):
    # Adım 2: StockCode’u POST olan gözlem birimlerini drop ediniz. (POST her faturaya eklenen bedel, ürünü ifade etmemektedir.)
    drop_list = df[df['StockCode'] == 'POST'].index
    # Adım 3: Boş değer içeren gözlem birimlerini drop ediniz.
    df.drop(drop_list, inplace = True) 
    dataframe.dropna(inplace=True)
    # Adım 4: Invoice içerisinde C bulunan değerleri veri setinden çıkarınız. (C faturanın iptalini ifade etmektedir.)
    dataframe = dataframe[~dataframe["InvoiceNo"].str.contains("C", na=False)]
    # Adım 5: Price değeri sıfırdan küçük olan gözlem birimlerini filtreleyiniz.
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["UnitPrice"] > 0]
    # Adım 6: Price ve Quantity değişkenlerinin aykırı değerlerini inceleyiniz, gerekirse baskılayınız.
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "UnitPrice")
    return dataframe

df = retail_data_prep(df)

# Görev 2:  Alman Müşteriler Üzerinden Birliktelik Kuralları Üretme

In [5]:
# Adım 1: Aşağıdaki gibi fatura ürün pivot table’i oluşturacak create_invoice_product_df fonksiyonunu tanımlayınız.
# Description | NINE DRAWER OFFICE TIDY | SET 2 TEA TOWELS I LOVE LONDON | SPACEBOY BABY GIFT SET…
# Invoice
# 536370                  0                              1                               0
# 536852                  1                              0                               1
# 536974                  0                              0                               0
# 537065                  1                              0                               0
# 537463                  0                              0                               1

# Preparing ARL Data Structures (Invoice - Product matrix)
de_df = df[df['Country'] == "Germany"]

def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['InvoiceNo', "StockCode"])['Quantity'].sum().unstack().fillna(False). \
            applymap(lambda x: True if x > 0 else False)
    else:
        return dataframe.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: True if x > 0 else False)

de_inv_pro_df = create_invoice_product_df(de_df)
de_inv_pro_df.iloc[0:5, 0:5]


Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON
InvoiceNo,,,,,
536527,False,False,False,False,False
536840,False,False,False,False,False
536861,False,False,False,False,False
536967,False,False,False,False,False
536983,False,False,False,False,False


In [7]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)


check_id(df, '10002')
check_id(df, '10120')

['INFLATABLE POLITICAL GLOBE ']
['DOGGY RUBBER']


In [8]:
def create_rules(dataframe, id=True, country="Germany"):
    dataframe = dataframe[dataframe['Country'] == country]
    dataframe = create_invoice_product_df(dataframe, id)
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    return rules

df = df_.copy()

df = retail_data_prep(df)
rules = create_rules(df)

# Görev 3: Sepet İçerisindeki Ürün Id’leri Verilen Kullanıcılara Ürün Önerisinde Bulunma

In [9]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

print(arl_recommender(rules, '21987', 1))
print(arl_recommender(rules, '23235', 2))
print(arl_recommender(rules, '22747', 3))

['21086']
['23245', '23243']
['22748', '22746', '22746']
